In [80]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [81]:
data = pd.read_csv("jester-data-1.csv", header = None)
d = data.to_latex()

In [82]:
text_file = open("Output.txt", "w")
text_file.write(d)
text_file.close()

In [83]:
training = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8+1):int(len(data)*0.9)]
validation = data[int(len(data)*0.9+1):]

In [84]:
n_features = 2

user_ratings = training.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [85]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 20):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not rating == 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if(iteration%10000 == 0 ):
        print mse
                    
                    
    


In [86]:
#train(0,0,10)
sgd()
#user_preferences 
#item_features

26.0861389955
25.086840742
24.7532256808
24.5079526556
24.2744786325
24.0409717295
23.8060243422
23.5705204043
23.3359877955
23.1041178218
22.8765616108
22.6548126948
22.4401261675
22.2334593723
22.0354281292
21.8462744665
21.6658419085
21.4935543605
21.3283953089
21.1688859077


In [87]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 1.057,  0.863,  0.808, ...,  1.169,  0.678,  1.503],
       [ 1.112,  1.189,  1.189, ...,  1.73 ,  1.343,  1.756],
       [ 2.937,  2.538,  2.415, ...,  3.498,  2.198,  4.263],
       ..., 
       [ 3.093,  2.648,  2.514, ...,  3.641,  2.261,  4.474],
       [ 2.052,  1.681,  1.576, ...,  2.28 ,  1.33 ,  2.921],
       [ 1.615,  1.429,  1.369, ...,  1.984,  1.285,  2.365]])

In [88]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,(-7.820|1.057),(8.790|0.863),(-9.660|0.808),(-8.160|-0.003),(-7.520|0.794),(-8.500|1.721),(-9.850|0.234),(4.170|-0.025),(-8.980|0.368),(-4.760|1.260),...,(2.820|1.598),(99.000|1.328),(99.000|1.924),(99.000|1.145),(99.000|1.117),(99.000|1.431),(-5.630|1.429),(99.000|1.169),(99.000|0.678),(99.000|1.503)
1,(4.080|1.112),(-0.290|1.189),(6.360|1.189),(4.370|0.670),(-2.380|1.001),(-9.660|1.910),(-0.730|0.530),(-5.340|0.232),(8.880|0.966),(9.220|1.370),...,(2.820|1.975),(-4.950|1.779),(-0.290|2.018),(7.860|1.771),(-0.190|1.593),(-2.140|1.819),(3.060|1.913),(0.340|1.730),(-4.320|1.343),(1.070|1.756)
2,(99.000|2.937),(99.000|2.538),(99.000|2.415),(99.000|0.329),(9.030|2.290),(9.270|4.831),(9.030|0.792),(9.270|0.059),(99.000|1.312),(99.000|3.522),...,(99.000|4.587),(99.000|3.881),(99.000|5.341),(9.080|3.465),(99.000|3.312),(99.000|4.132),(99.000|4.175),(99.000|3.498),(99.000|2.198),(99.000|4.263)
3,(99.000|1.663),(8.350|1.498),(99.000|1.441),(99.000|0.331),(1.800|1.332),(8.160|2.757),(-2.820|0.510),(6.210|0.089),(99.000|0.868),(1.840|2.004),...,(99.000|2.661),(99.000|2.280),(99.000|3.023),(0.530|2.084),(99.000|1.966),(99.000|2.407),(99.000|2.453),(99.000|2.089),(99.000|1.381),(99.000|2.452)
4,(8.500|2.536),(4.610|2.176),(-4.170|2.067),(-5.390|0.248),(1.360|1.968),(1.600|4.166),(7.040|0.669),(4.610|0.037),(-0.440|1.102),(5.730|3.039),...,(5.190|3.945),(5.580|3.331),(4.270|4.612),(5.190|2.962),(5.730|2.838),(1.550|3.551),(3.110|3.583),(6.550|2.994),(1.800|1.865),(1.600|3.672)
5,(-6.170|-1.275),(-3.540|-1.189),(0.440|-1.154),(-8.500|-0.352),(-7.090|-1.046),(-4.320|-2.129),(-8.690|-0.432),(-0.870|-0.106),(-6.650|-0.750),(-1.800|-1.543),...,(-3.540|-2.083),(-6.890|-1.804),(-0.680|-2.318),(-2.960|-1.680),(-2.180|-1.568),(-3.350|-1.892),(0.050|-1.940),(-9.080|-1.675),(-5.050|-1.150),(-3.450|-1.906)
6,(99.000|2.430),(99.000|2.295),(99.000|2.233),(99.000|0.738),(8.590|2.009),(-9.850|4.067),(7.720|0.852),(8.790|0.228),(99.000|1.487),(99.000|2.946),...,(99.000|4.000),(99.000|3.476),(99.000|4.416),(99.000|3.259),(99.000|3.031),(2.330|3.637),(99.000|3.739),(99.000|3.242),(99.000|2.256),(99.000|3.649)
7,(6.840|0.213),(3.160|0.235),(9.170|0.237),(-6.210|0.147),(-8.160|0.196),(-1.700|0.368),(9.270|0.109),(1.410|0.052),(-5.190|0.201),(-4.420|0.263),...,(7.230|0.386),(-1.120|0.351),(-0.100|0.386),(-5.680|0.354),(-3.160|0.316),(-3.350|0.356),(2.140|0.377),(-0.050|0.345),(1.310|0.275),(0.000|0.341)
8,(-3.790|0.828),(-3.540|0.659),(-9.420|0.612),(-6.890|-0.043),(-8.740|0.612),(-0.290|1.341),(-5.290|0.166),(-8.930|-0.035),(-7.860|0.253),(-1.600|0.984),...,(4.370|1.233),(-0.290|1.017),(4.170|1.506),(-0.290|0.863),(-0.290|0.849),(-0.290|1.101),(-0.290|1.094),(-0.290|0.885),(-3.400|0.493),(-4.950|1.166)
9,(3.010|2.205),(5.150|1.883),(5.150|1.787),(3.010|0.195),(6.410|1.706),(5.150|3.620),(8.930|0.573),(2.520|0.024),(3.010|0.940),(8.160|2.641),...,(99.000|3.421),(4.470|2.884),(99.000|4.011),(99.000|2.557),(99.000|2.455),(99.000|3.078),(99.000|3.103),(99.000|2.588),(99.000|1.602),(99.000|3.187)


In [89]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"(-7.82, 1.05732228996)","(8.79, 0.862805880715)","(-9.66, 0.808097882636)","(-8.16, -0.00283370846554)","(-7.52, 0.79436856769)","(-8.5, 1.72108330889)","(-9.85, 0.23411935749)","(4.17, -0.0252563663176)","(-8.98, 0.368092356149)","(-4.76, 1.26002290623)",...,"(2.82, 1.59803156532)","(99.0, 1.32836650694)","(99.0, 1.92359828802)","(99.0, 1.14518114544)","(99.0, 1.11705144555)","(99.0, 1.43093986202)","(-5.63, 1.4289574818)","(99.0, 1.16901124429)","(99.0, 0.677833763946)","(99.0, 1.50291956904)"
1,"(4.08, 1.11173164099)","(-0.29, 1.1887020229)","(6.36, 1.1893393064)","(4.37, 0.669568005912)","(-2.38, 1.00107610913)","(-9.66, 1.91003599993)","(-0.73, 0.52952135562)","(-5.34, 0.231881612949)","(8.88, 0.966069334827)","(9.22, 1.36967337661)",...,"(2.82, 1.97524032577)","(-4.95, 1.77887033726)","(-0.29, 2.01789251407)","(7.86, 1.77068629272)","(-0.19, 1.59313752706)","(-2.14, 1.81850350615)","(3.06, 1.91305466047)","(0.34, 1.73044946557)","(-4.32, 1.34315743293)","(1.07, 1.75608275917)"
2,"(99.0, 2.93696448427)","(99.0, 2.53755285053)","(99.0, 2.41470080372)","(99.0, 0.328774607694)","(9.03, 2.28955448899)","(9.27, 4.83096759148)","(9.03, 0.792156222344)","(9.27, 0.0592062587436)","(99.0, 1.31230213715)","(99.0, 3.52244363458)",...,"(99.0, 4.58656386724)","(99.0, 3.88113913483)","(99.0, 5.34090313615)","(9.08, 3.46461419156)","(99.0, 3.31240961177)","(99.0, 4.13191402781)","(99.0, 4.17477873931)","(99.0, 3.49812523183)","(99.0, 2.19841487935)","(99.0, 4.26272614094)"
3,"(99.0, 1.66306602)","(8.35, 1.49767975214)","(99.0, 1.44067263404)","(99.0, 0.331389021464)","(1.8, 1.33227457351)","(8.16, 2.75722752322)","(-2.82, 0.509744182217)","(6.21, 0.0893286903826)","(99.0, 0.868123196165)","(1.84, 2.00427117257)",...,"(99.0, 2.66084940783)","(99.0, 2.28022280717)","(99.0, 3.02329096865)","(0.53, 2.08418661722)","(99.0, 1.96604962789)","(99.0, 2.40749489398)","(99.0, 2.45263313498)","(99.0, 2.08906397131)","(99.0, 1.38098764785)","(99.0, 2.45174758865)"
4,"(8.5, 2.53611687078)","(4.61, 2.17619959683)","(-4.17, 2.06701178314)","(-5.39, 0.248019556815)","(1.36, 1.9682201742)","(1.6, 4.16626357351)","(7.04, 0.668921940074)","(4.61, 0.0373370461411)","(-0.44, 1.10230087136)","(5.73, 3.03929691544)",...,"(5.19, 3.9448350157)","(5.58, 3.33103830315)","(4.27, 4.61220762117)","(5.19, 2.96152234403)","(5.73, 2.83798598302)","(1.55, 3.55122621722)","(3.11, 3.58308476218)","(6.55, 2.99394329001)","(1.8, 1.8647309517)","(1.6, 3.67155296047)"
5,"(-6.17, -1.27524723836)","(-3.54, -1.1892430338)","(0.44, -1.15396251803)","(-8.5, -0.351624891222)","(-7.09, -1.04563873436)","(-4.32, -2.12881094122)","(-8.69, -0.431958524892)","(-0.87, -0.105969128319)","(-6.65, -0.749636733395)","(-1.8, -1.54338216924)",...,"(-3.54, -2.08312150784)","(-6.89, -1.8038943994)","(-0.68, -2.31759395318)","(-2.96, -1.68031349754)","(-2.18, -1.56827075258)","(-3.35, -1.89159610347)","(0.05, -1.94021973103)","(-9.08, -1.67458646962)","(-5.05, -1.15035741948)","(-3.45, -1.90550521198)"
6,"(99.0, 2.43039316961)","(99.0, 2.2948254984)","(99.0, 2.23344272491)","(99.0, 0.737845984946)","(8.59, 2.00949609246)","(-9.85, 4.06723980744)","(7.72, 0.851763700104)","(8.79, 0.227977611266)","(99.0, 1.48697057956)","(99.0, 2.94592187131)",...,"(99.0, 3.99975498968)","(99.0, 3.47637449348)","(99.0, 4.41644707232)","(99.0, 3.25941953169)","(99.0, 3.03098838156)","(2.33, 3.63665003767)","(99.0, 3.7390472056)","(99.0, 3.24193114515)","(99.0, 2.25584823228)","(99.0, 3.64925400315)"
7,"(6.84, 0.212505591842)","(3.16, 0.235149441516)","(9.17, 0.236909915471)","(-6.21, 0.146935630356)","(-8.16, 0.196026186464)","(-1.7, 0.367928461413)","(9.27, 0.109200533146)","(1.41, 0.0516051875649)","(-5.19, 0.200976660446)","(-4.42, 0.263073228145)",...,"(7.23, 0.385874631316)","(-1.12, 0.350794892422)","(-0.1, 0.385584451833)","(-5.68, 0.354426737546)","(-3.16, 0.316319535888)","(-3.35, 0.356448663599)","(2.14, 0.37724463635)","(-0.05, 0.344895917415)","(1

In [102]:
np.amax(predictions)


10.748057269086427

In [103]:
np.amin(predictions)

-9.7038502198515069

In [ ]:
d = comparison_data.to_latex()
text_file = open("comparison.txt", "w")
text_file.write(d)
text_file.close()